# DD2424 Project in Deep Learning in Data Science

## Imports

In [1]:
from functions import *
import pandas as pd
import numpy as np
import plotly.express as px
import tensorflow as tf
import os

2023-05-18 10:33:03.195069: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Vanilla RNN

### Load and Preprocess Data

In [ ]:
training_data_filename = '../Dataset/Training/edgar.txt'
book_data = np.array(load_data(training_data_filename))
book_chars = np.unique(book_data)

char_to_ind = {ch:i for i,ch in enumerate(book_chars)}
ind_to_char = {i:ch for i,ch in enumerate(book_chars)}
k = book_chars.shape[0]
m = 100
eta = 0.1
seq_length = 25
sig = 0.01


### Train

In [ ]:
rnn = RNN(m, k, eta, seq_length, sig)
h0 = np.zeros((m, 1))
max_iter = 200000
epochs = 10
smoothloss_list = []
loss_list = []
iteration = 0
smoothloss = 0
sentences = []
for i in range(epochs):
    rnn.hprev = np.zeros((m, 1))
    for e in range(0, book_data.shape[0]-seq_length-1, seq_length):
        X_chars = book_data[e:e+seq_length]
        Y_chars = book_data[e+1:e+seq_length+1]
        X = one_hot_encoding(X_chars, char_to_ind, k)
        Y = one_hot_encoding(Y_chars, char_to_ind, k)
        loss = rnn.adagrad(X, Y, h0, iteration)
        if smoothloss == 0:
            smoothloss = loss
        smoothloss = 0.999*smoothloss + 0.001*loss
     
        if iteration % 10000 == 1:
            print('Iteration: {}, Loss: {} '.format(iteration, smoothloss))
            y = rnn.synthetize(rnn.hprev, X[:, 0], 200)
            sentence = one_hot_decoding(y, ind_to_char)
            print(sentence + "\n")
            #sentences.append(sentence)
            smoothloss_list.append(smoothloss)
            loss_list.append(loss)
        
        iteration += 1
        if iteration>max_iter:
            break


### Evaluate and plot

In [ ]:

with open(training_data_filename,encoding='cp850',mode='r') as file:
    validation_text = file.read()
start_char = "T"
start_char_onehot = one_hot_encoding(start_char, char_to_ind, k)
generated_text_vanilla_onehot = rnn.synthetize(rnn.hprev, start_char_onehot, 1000)
generated_text_vanilla = start_char + one_hot_decoding(generated_text_vanilla_onehot, ind_to_char)
print(generated_text_vanilla)

# Calculate performance metrics for generated text
nmax = 4
fraction_correct_words, bleu_score = measure_bleu(text_generated=generated_text_vanilla, text_val=validation_text, n_max=nmax)
repetition_score = measure_diversity(text_generated=generated_text_vanilla, n_max=nmax)
print("\n loss function", loss_list)
print("\n fraction of correctly spelled words: {} \n Bleu score: {} \n Repetition score: {}".format(fraction_correct_words, bleu_score, repetition_score))



fig = px.line(smoothloss_list, title='Smoothed loss over epochs', width=600)
fig.update_layout(showlegend=False)
fig.update_yaxes(title_text="smoothed loss")
fig.update_xaxes(title_text="iteration step, in multiples of 10k")
fig.show()



## LSTM

### Load and Preprocess Data

In [2]:

training_data_filename = '../Dataset/Training/edgar.txt'
encoding = "utf8"
book_data = np.array(load_data(training_data_filename,remove_footnotes=False, encoding=encoding))

with open(training_data_filename,encoding=encoding,mode='r') as f:
   words = f.read().split()
with open(training_data_filename,encoding=encoding,mode='r') as file:
    validation_text = file.read()
    # takes a little time due to synonym swap
    book_data_augmented = augment_data(data=validation_text, n_synonyms=300, n_word_swaps=500, n_deletions=100, n_sentence_shuffles=100)

book_chars = np.unique(book_data)
char_to_ind = {ch:i for i,ch in enumerate(book_chars)}
ind_to_char = {i:ch for i,ch in enumerate(book_chars)}
k = book_chars.shape[0]

vocab_size = len(book_chars)
embedding_dim = 256 


def create_batches(data, batch_size, seq_length):
    book_data_ind = np.array([char_to_ind[c] for c in data])

    # Split data into sequences
    char_dataset = tf.data.Dataset.from_tensor_slices(book_data_ind)
    sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

    # Split data into X, Y
    dataset = sequences.map(split_input_target)

    # This organizes the data into groups of sequences. batch_size denotes the number of sequences in a batch, and seq_length denotes the number of characters in a sequence.
    dataset = dataset.shuffle(10000).batch(batch_size, drop_remainder=True)
    
    return dataset

### Functions for training

In [28]:

def build_model(rnn_units, batch_size, nr_lstm_layers, bn):
    m = tf.keras.Sequential()
    m.add(tf.keras.layers.Embedding(input_dim=vocab_size, output_dim=embedding_dim, batch_input_shape=[batch_size, None]))
    for i in range(nr_lstm_layers):
        m.add(tf.keras.layers.LSTM(units=rnn_units, return_sequences=True, stateful=True, recurrent_initializer=tf.keras.initializers.GlorotNormal()))
    if bn:
        m.add(tf.keras.layers.BatchNormalization())
    m.add(tf.keras.layers.Dense(vocab_size))
    return m


def train_model(dataset, eta, rnn_units, batch_size, output_filename, nr_lstm_layers=1, bn=False):
    m = build_model(rnn_units, batch_size, nr_lstm_layers, bn)
    for i_ex, t_ex in dataset.take(1):
        example_pred = m(i_ex)  # this step builds the model

    # Specify update rule and compile model
    adam_opt = tf.keras.optimizers.Adam(learning_rate=eta)
    loss_func = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
    m.compile(optimizer=adam_opt, loss=loss_func)

    # Configure checkpoints
    current_dir_path = os.getcwd()
    checkpoint_dir = os.path.join(os.path.join(os.path.join(current_dir_path, "tmp"), output_filename), "training_checkpoints")
    os.makedirs(checkpoint_dir, exist_ok=True)
    checkpoint_prefix = os.path.join(checkpoint_dir, 'ckpt_{epoch:02d}.hdf5')
    checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_prefix,save_weights_only=True, save_best_onl =False)

    # train
    history = m.fit(x=dataset, epochs=n_epochs, callbacks=[checkpoint_callback])
    return m, history

### Grid search

In [ ]:

rnn_units = 200
n_epochs = 5
modelpathname = "v6_1"
checkpoint_dir = os.path.join(os.path.join(os.path.join(os.getcwd(),"tmp"), modelpathname), "training_checkpoints")
output_str = "batch_size | seq_len | eta | loss | BLEU2 \n"

batch_sizes = [25, 75, 125]
seq_lengths = [25, 75, 125]
learningrates = [0.1, 0.01, 0.001]
count = 0
for batch_size in batch_sizes:
    for seq_length in seq_lengths:
        dataset = create_batches(batch_size, seq_length)
        for eta in learningrates:
            print(count/27)
            model, history = train_model(dataset, eta, rnn_units, batch_size)
            final_loss = history.history['loss'][-1]
            # generate text
            m = build_model(rnn_units, 1)
            m.load_weights(checkpoint_dir + "/" + "ckpt_0{}.hdf5".format(n_epochs))
            m.build(tf.TensorShape([1, None]))
            gen_text = generate_text(model=m, start_string="The ", text_size=1000, 
                                    char_to_ind=char_to_ind, ind_to_char=ind_to_char, temp=1.0, p=None)
            # Measure performance
            frac_corr_words, bleu2 = measure_bleu(gen_text, validation_text, 2)
            output_str += "{}      {}      {}      {}      {} \n".format(batch_size, seq_length, eta, final_loss, bleu2)

            count += 1

print(output_str)  
            

### Build and train one model

In [29]:

# Number of RNN units.
rnn_units = 500
n_epochs = 5
eta = 0.01
batch_size = 50
seq_length = 50


modelpathname = 'v4_1'
checkpoint_dir = os.path.join(os.path.join(os.path.join(os.getcwd(),"tmp"), modelpathname), "training_checkpoints")
dataset = create_batches(book_data, batch_size, seq_length)
nr_layers = 1
model, history = train_model(dataset, eta, rnn_units, batch_size, modelpathname, nr_layers, False)
validation_data = validation_text


Epoch 1/5
223/223 [==============================] - 100s 435ms/step - loss: 2.2253
Epoch 2/5
223/223 [==============================] - 99s 439ms/step - loss: 1.6882
Epoch 3/5
223/223 [==============================] - 110s 489ms/step - loss: 1.5465
Epoch 4/5
223/223 [==============================] - 110s 487ms/step - loss: 1.4732
Epoch 5/5
223/223 [==============================] - 119s 527ms/step - loss: 1.4279


### Generate text, evauate and plot

In [31]:
checkpoint_dir = os.path.join(os.path.join(os.path.join(os.getcwd(),"tmp"), modelpathname), "training_checkpoints")
plot_title = 'Loss over epochs'

plotbool = True
outputfile_bool = True

# Find latest checkpoint file, because tf.train.latest_checkpoint(dir) doesn't work for some reason
latest_epoch = 0
latest_checkpoint_file = ""
for file in os.listdir(checkpoint_dir):
    if file != ".DS_Store":
        e = int(file.split("_")[1].split(".")[0])
        if e>latest_epoch:
            latest_epoch = e
            latest_checkpoint_file = file
print(checkpoint_dir + "/" + latest_checkpoint_file)


m = build_model(rnn_units=rnn_units, batch_size=1, nr_lstm_layers=1, bn=False)
m.load_weights(checkpoint_dir + "/" + latest_checkpoint_file)
m.build(tf.TensorShape([1, None]))

nucleus_probability = None
temp = 1.0
gen_text = generate_text(model=m, start_string="The ", text_size=1000, char_to_ind=char_to_ind, ind_to_char=ind_to_char, temp=temp, p=nucleus_probability)


if plotbool:
    d = get_n_grams(gen_text, 1)
    fig = px.line(history.history['loss'], title=plot_title, width=600)
    fig.update_layout(showlegend=False)
    fig.update_xaxes(title_text="Iteration step, in multiples of 10k")
    fig.update_yaxes(title_text="smoothed loss")
    fig.show()
    

# Calculate performance metrics for generated text
fraction_correct_words, bleu_score2 = measure_bleu(text_generated=gen_text, text_val=validation_data, n_max=2)
repetition_score2 = measure_diversity(text_generated=gen_text, n_max=2)

output_str = gen_text + "\n ------------ \n loss {} \n fraction of correctly spelled words: {} \n Bleu score2: {}, Repetition score2: {} ".format(history.history['loss'], fraction_correct_words, bleu_score2, repetition_score2)
output_str += "\n settings: batch_size, seq_length, eta, rnn_units, n_epochs = {}, {}, {}, {}, {}".format(batch_size, seq_length, eta, rnn_units, n_epochs)
print(output_str)


# Create output file
output_path = os.path.join(os.path.join(os.getcwd(), "tmp"), modelpathname)
if outputfile_bool:
    with open(os.path.join(output_path, "perf.txt"), "w") as file:
        file.write(output_str)   
    




/Users/sofia/Documents/Skola/KTH/Master/Djupinlärning/Projektt/DD2424_Project/src/tmp/v4_1/training_checkpoints/ckpt_05.hdf5


The which short distancer, inquisible sight, and to pan is the losteys. But, than Endand Sir
       “But You girl, “who had been atmosphere nunor. He was soul that, slupsure in the
      put and this has lose is caude, they gentlerely have been a character
      de you will at haighing and
      tesheaving and tow. This proper when Magazin upon no ob—be kind so object, then
      more Peotne. If is
      noternate into be so, so disposited, and his closing upwant, screys’
      take to
      we said, at the
      wind, liken about one of probable. It in the same
      adverthy far a view which tart of his
      diminds. Nove, being to uggles of apparence, the same thought him. But,
      we respeciety of my
      bitch startlest Mrs. B. was the murders was trallyion were knowledge. (*15) Them what a few so were leaves of gitty if!—nd there onathing que) se, one and repeation; and this eas
      peculiay of the tempeate the body we as
      manyster. Weits and in period, at implaions. T